# Python library for loading a log of the Consumer Physics SCIO


In [ ]:
import json
import struct
import datetime
import os
import csv
import base64
import numpy as np
import re

In [ ]:
class scio_log:
    def __init__(self):
        self.device_info = {}
        self.scan_rawdata = {}
        self.b64_data = {}
        self.spectral_data = {}
        self.calibration_data = {}
        self.temperature = {}
        self.key = b''
        self.cmd_info = r"DeviceInfo{(.*?)}"
        self.cmd_scan = r"ResponseCommandParse	bytes:( [\s\w]+)"
        self.cmd_b64  = r'^{"device_id"(.*)$' #r"({\"device_id\".*?\"})"
        self.cmd_spec = r"Response: {\"_type\":\"POST spectroscan2\",\"spectrum\":(.*?)}"
    
    def load_log(self, log_fn, out_path, silent=True):
        next_task = 'dev_info'
        
        scan_data_started = True
        scan_data = ''
        end_of_scan = False
        
        scan_nb = 0
        i = 0
        
        with open(log_fn, "r") as file:
            for line in file:
                sample_done = False
                # Search for the pattern in each line
                bytes_match = re.search(self.cmd_scan, line)
                info_match  = re.search(self.cmd_info, line)
                b64_match   = re.search(self.cmd_b64, line)
                spec_match  = re.search(self.cmd_spec, line)
                if bytes_match:
                    # Extract the bytes portion from the matched line
                    hex_str = bytes_match.group(1).replace(" 0x", "")
                    if('01BA04' in hex_str):
                        print('Detected scan ' + str(scan_nb) + ':')
                        scan_nb += 1
                        self.temperature = self.read_temperature(hex_str.strip())
                        print('  - Temperature: ' + str(self.temperature))
                    elif('01BA02' in hex_str):
                        scan_data = hex_str.strip()[10:]
                        scan_data_started = False
                    elif((~scan_data_started) and (hex_str[0:2] != '01')):
                        scan_data_started = True
                        scan_data = scan_data + hex_str.strip()[2:]
                        if(len(hex_str.strip()) < 40):
                            scan_data_started = True
                            if(i == 0):
                                sample = scan_data
                                print('  - Sample bytes: ', str(int(len(sample)/2)))
                                self.scan_rawdata.update({'sample_dark': sample})
                            elif(i == 1):
                                sample_dark = scan_data
                                print('  - Sample bytes: ', str(int(len(sample_dark)/2)))
                                self.scan_rawdata.update({'sample': sample_dark})
                            elif(i == 2):
                                sample_gradient = scan_data
                                print('  - Sample bytes: ', str(int(len(sample_gradient)/2)))
                                self.scan_rawdata.update({'sample_gradient': sample_gradient})
                                i = 0
                            i += 1
                    else:
                        print("Bytes: {}".format(hex_str.strip()))
                if info_match:
                    self.parse_device_info(info_match.group(1))
                if b64_match:
                    self.b64_data = json.loads(b64_match.group())
                if spec_match:
                    self.spectral_data = spec_match.group(1)
                    print('  - Saving scan data')
                    self.write_data_file('scan_from_log', out_path)      
        return
    
    def parse_device_info(self, match_str):
        info = str(match_str).replace("\'", "").split(", ")
        for item in info:
            key, value = item.split("=")
            if value.isdigit():
                self.device_info[key] = int(value)
            elif value.lower() == "true":
                self.device_info[key] = True
            elif value.lower() == "false":
                self.device_info[key] = False
            else:
                self.device_info[key] = value
        return(self.device_info)
    
    def read_temperature(self, hex_string):
        response_data_raw = bytearray.fromhex(hex_string[10:])
        response_length = len(response_data_raw)
        # Parse response
        num_vars = response_length / 4 # divide by 4 because we are dealing with longs
        data_struct = '<' + str(int(num_vars)) + 'L' # This is usually '<3l' or '<lll'
        # Convert bytes
        response_data = struct.unpack(data_struct, response_data_raw)
        # Convert to actual temperatures
        cmosTemperature   = (response_data[0] - 375.22) / 1.4092 # From the disassembled Android app
        chipTemperature   = response_data[1] / 100
        objectTemperature = response_data[2] / 100
        temperatures = {'cmos_t': cmosTemperature, 'chip_t': chipTemperature, 'obj_t': objectTemperature}
        return(temperatures)
    
    # Saves a single scan as raw data to JSON
    def write_data_file(self, output_fn_base, out_path):
        # Check if output path exists
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        
        # Create file name
        timestamp = self.b64_data['sampled_at'][:19].replace('T', '_').replace('-', '').replace(':', '')
        output_fn = output_fn_base + '_' + timestamp + '.json'
        
        # Create the json string
        jsondata = {}
        jsondata['device'] = self.device_info
        jsondata['temperature'] = self.temperature
        jsondata['raw_data'] = self.scan_rawdata
        jsondata['b64_data'] = self.b64_data
        jsondata['spec_data'] = self.spectral_data
        
        # Write data to JSON file
        with open(out_path + output_fn, 'w') as outfile:
            json.dump(jsondata, outfile, indent=4)
        return
    
    def encode_b64(self, bytestring):
        urlSafeEncodedBytes = base64.urlsafe_b64encode(bytestring)
        urlSafeEncodedStr = str(urlSafeEncodedBytes, 'utf-8')
        return(urlSafeEncodedStr)

In [ ]:
log_fn  = './01_rawdata/log_files/log_20200604-bark.txt'
#log_fn  = './01_rawdata/log_files/log_20200604-rock.txt'
log_fn = './01_rawdata/log_files/log_20211020_skin.txt'
output_path = './01_rawdata/log_extracted/'

# Now set up device
scio = scio_log() # Create an instance for this device address
scio.load_log(log_fn, output_path, silent=False)
print('Done...')